In [14]:
from line_solver import *
import numpy as np
GlobalConstants.set_verbose(VerboseLevel.STD)

In [15]:
# Model parameters
N = 4  # number of jobs in Class1
c = 2  # number of servers

# Create network
model = Network('model')
node1 = Delay(model, 'Delay')
node2 = Queue(model, 'Queue1', SchedStrategy.PS)

# Create job classes
jobclass1 = ClosedClass(model, 'Class1', N, node1, 0)
jobclass2 = ClosedClass(model, 'Class2', N // 2, node1, 0)  # N/2 = 2 jobs

# Set service times
node1.set_service(jobclass1, Exp.fit_mean(1.0))   # Class1: mean = 1.0
node1.set_service(jobclass2, Exp.fit_mean(2.0))   # Class2: mean = 2.0
node2.set_service(jobclass1, Exp.fit_mean(1.5))   # Class1: mean = 1.5
node2.set_service(jobclass2, Exp.fit_mean(2.5))   # Class2: mean = 2.5
node2.set_number_of_servers(c)

# Create routing matrix
P = model.init_routing_matrix()
P.set(jobclass1, jobclass1, node1, node2, 1.0)
P.set(jobclass1, jobclass1, node2, node1, 1.0)
P.set(jobclass2, jobclass2, node1, node2, 1.0)
P.set(jobclass2, jobclass2, node2, node1, 1.0)
model.link(P)

In [16]:
avg_table_mva=MVA(model, method='exact').avg_table()

MVA [method: exact, lang: java, env: 17.0.15] completed in 0.000020s.
  Station JobClass    QLen    Util   RespT  ResidT    ArvR    Tput
0   Delay   Class1  0.8975  0.8975  1.0000  1.0000  0.8975  0.8975
1   Delay   Class2  0.5116  0.5116  2.0000  2.0000  0.2558  0.2558
2  Queue1   Class1  3.1025  0.6731  3.4568  3.4568  0.8975  0.8975
3  Queue1   Class2  1.4884  0.3197  5.8190  5.8190  0.2558  0.2558


In [17]:
# Create load-dependent model
ldmodel = Network('ldmodel')
ldnode1 = Delay(ldmodel, 'Delay')
ldnode2 = Queue(ldmodel, 'Queue1', SchedStrategy.PS)

# Create job classes for load-dependent model
ldjobclass1 = ClosedClass(ldmodel, 'Class1', N, ldnode1, 0)
ldjobclass2 = ClosedClass(ldmodel, 'Class2', N // 2, ldnode1, 0)

# Set service times for load-dependent model
ldnode1.set_service(ldjobclass1, Exp.fit_mean(1.0))
ldnode1.set_service(ldjobclass2, Exp.fit_mean(2.0))
ldnode2.set_service(ldjobclass1, Exp.fit_mean(1.5))
ldnode2.set_service(ldjobclass2, Exp.fit_mean(2.5))

# Set load dependence: min(total_jobs+1, 2) servers available
# Create load dependence matrix where rows are total jobs (0 to 6) and columns are service rates
max_jobs = N + N // 2  # 4 + 2 = 6 total jobs
ld_matrix = np.zeros((max_jobs + 1, 2))  # (total_jobs+1) x num_classes

for total_jobs in range(max_jobs + 1):
    servers_available = min(total_jobs + 1, c)
    ld_matrix[total_jobs, 0] = servers_available  # Class1 scaling
    ld_matrix[total_jobs, 1] = servers_available  # Class2 scaling

ldnode2.set_load_dependence(ld_matrix)
ldnode2.set_number_of_servers(c)

# Create routing matrix for load-dependent model
P_ld = ldmodel.init_routing_matrix()
P_ld.set(ldjobclass1, ldjobclass1, ldnode1, ldnode2, 1.0)
P_ld.set(ldjobclass1, ldjobclass1, ldnode2, ldnode1, 1.0)
P_ld.set(ldjobclass2, ldjobclass2, ldnode1, ldnode2, 1.0)
P_ld.set(ldjobclass2, ldjobclass2, ldnode2, ldnode1, 1.0)
ldmodel.link(P_ld)

In [ ]:
# Solve load-dependent model with CTMC
try:
    lldAvgTableCTMC=CTMC(ldmodel).avg_table()
    print(lldAvgTableCTMC)
except Exception as e:
    print(f"CTMC solver error: {type(e).__name__}")

In [ ]:
# Solve with NC methods - may fail on multi-station load-dependent models
try:
    lldAvgTableNC=NC(ldmodel).avg_table()
    print(lldAvgTableNC)
except Exception as e:
    print(f"NC solver limitation: {type(e).__name__}")

In [ ]:
try:
    lldAvgTableRD=NC(ldmodel, method='rd').avg_table()
    print(lldAvgTableRD)
except Exception as e:
    print(f"NC(rd) solver limitation: {type(e).__name__}")

In [ ]:
try:
    lldAvgTableNRP=NC(ldmodel, method='nrp').avg_table()
    print(lldAvgTableNRP)
except Exception as e:
    print(f"NC(nrp) solver limitation: {type(e).__name__}")

In [ ]:
try:
    lldAvgTableNRL=NC(ldmodel, method='nrl').avg_table()
    print(lldAvgTableNRL)
except Exception as e:
    print(f"NC(nrl) solver limitation: {type(e).__name__}")

In [ ]:
# Solve with MVA methods
try:
    lldAvgTableMVALD=MVA(ldmodel, method='exact').avg_table()
    print(lldAvgTableMVALD)
except Exception as e:
    print(f"MVA(exact) solver error: {type(e).__name__}")

In [ ]:
try:
    lldAvgTableQD=MVA(ldmodel, method='qd').avg_table()
    print(lldAvgTableQD)
except Exception as e:
    print(f"MVA(qd) solver error: {type(e).__name__}")